In [8]:
import tweepy
import pandas as pd
import time

In [13]:
API_KEY = "XXXXXXXXXX"
API_KEY_SECRECT = "XXXXXXXXXXXXX"
ACCESS_TOKEN = "XXXXXXXXXXXXXXXXXX"
ACCESS_TOKEN_SECRET = "XXXXXXXXXXXX"
BEARER_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [14]:
client = tweepy.Client(BEARER_TOKEN, wait_on_rate_limit=True)

In [1]:
query = "your_query_here"
# check for the length of query as it is restricted by the API
len(query)

In [2]:
esg_tweets = []
count = 0

for response in tweepy.Paginator(client.search_all_tweets,
                 query = query,
                 user_fields = ['username', 'public_metrics', 'id', 'location', 'name'],
                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text', 
                                 'in_reply_to_user_id', 'lang', 'referenced_tweets'],
                 expansions = ['author_id', 'in_reply_to_user_id'],
                 start_time = '2022-01-01T00:00:00Z',
                 end_time = '2022-05-01T00:00:00Z',
                 max_results=500):
    try:
        time.sleep(1)
        user_list = []
        user_dict = {}
        tweets_list = []
        
        # get user data
        for user in response.includes['users']:
            user_list.append({'id': user.id,
                                 'username': user.name,
                                 'description': user.description,
                                 'followers': user.public_metrics['followers_count'],
                                 'following': user.public_metrics['following_count'],
                                 'tweets_count': user.public_metrics['tweet_count'],
                                 'location' : user.location})

        tweet_count = 0
        
        # get tweets
        for tweet in response.data:

            tweets_list.append({'author_id': tweet.author_id,
                               'username': user_list[tweet_count]['username'],
                               'text': tweet.text,
                               'created_at': tweet.created_at,
                               'retweets': tweet.public_metrics['retweet_count'],
                                'replies': tweet.public_metrics['reply_count'],
                               'likes': tweet.public_metrics['like_count'],
                               'quote_count': tweet.public_metrics['quote_count'],
                               'in_reply_to_user_id': tweet.in_reply_to_user_id,
                               'geo': tweet.geo,
                               'referenced_tweets_id': tweet.referenced_tweets,
                               'referenced_tweets_type': tweet.referenced_tweets,
                               'lang': tweet.lang})
            tweet_count +=1

        user_df = pd.DataFrame(user_list)
        tweet_df = pd.DataFrame(tweets_list)
        
        # saving both user and tweets data
        user_df.to_csv(f"users_{count}.csv")
        tweet_df.to_csv(f"tweet_{count}.csv")
        count +=1
        
        # change this number to the number of tweets you wish to get
        if count*500 > 2500000:
            break
            
    except:
        print(f"tweets chunk {count} failed")          